In [1]:
import pandas as pd
import os
from collections import Counter
import pickle

In [2]:
PATH_TO_TXT = './txt/'
fnames = []
for i in os.listdir(PATH_TO_TXT):
    fnames.append(f'{PATH_TO_TXT}/{i}')

In [3]:
def parse_file(fname):
    """Parses txt files for RRUFF database"""
    _, r_id, _, _, id_ = fname.split('__')
    id_ = id_.replace('.txt', '').zfill(4)
    d = dict()
    x_data = []
    y_data = []

    with open(fname, 'r') as f:
        
        data = [i.replace('\n','') for i in f.readlines()[:-4]]
        
        d['ID'] = id_ # unique ID/primary key for db

        # categorical data
        cat_data = [i for i in data if i.startswith('#')] 
        for i in cat_data:
            name, value = i.split('=',1)
            name = name.replace('##','')
            if name == 'RRUFFID':
                d[name] = r_id
            else:
                d[name] = value
        
        # Assign None Values if key doesn't exists
        if not d.get("MEASURED CHEMISTRY", False):
            d["MEASURED CHEMISTRY"] = None
        if not d.get("IDEAL CHEMISTRY", False):
            d["IDEAL CHEMISTRY"] = None
        
        # numerical data
        last_cat_idx = data.index(cat_data[-1])
        num_data = data[last_cat_idx + 1:]
        for i in num_data:
            x, y = i.split(',')
            x_data.append(x)
            y_data.append(y)

        # Save CSV file with the x,y data
        df = pd.DataFrame({'x': x_data, 'y': y_data}, dtype=float)
        f_path = f"./CSV/{d['ID']}_{d['RRUFFID']}_{d['NAMES']}.csv"
        df.to_csv(f_path, index=False)

        # Add Path to the dictionary object
        d['CSV_PATH'] = f"./app/data/CSV/{d['ID']}_{d['RRUFFID']}_{d['NAMES']}.csv"

        return d


In [4]:
files = []
for i in fnames:
    files.append(parse_file(i))

In [5]:
# Check for duplicates in unique ID values
f_ids = [i['ID'] for i in files]
print([k for k,v in Counter(f_ids).items() if v > 1]) 

[]


In [6]:
# Save As pickle file
with open('./processed_data.pkl', 'wb') as f:
    pickle.dump(files, f)

In [2]:
# Load the pickle file
with open('data.pkl', 'rb') as f:
    files = pickle.load(f)